### GA4 API 연결 전 선행 작업
1. 공식 문서 접속: https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries?hl=ko
2. Google Analytics Data API v1 사용 설정 클릭
3. 이름 설정 후, Next 클릭
4. Download private key as JSON 클릭
5. client_email 복사

### 구글 클라우드 콘솔에서 선행 작업
1) [cloud.google.com](https://cloud.google.com/?hl=ko)에서 구글 클라우드 콘솔에 접속한다.
2) 메뉴에서 API 및 서비스 > 라이브러리를 클릭한다.
3) Google Analytics Data API를 검색하여 '사용'을 클릭한다. (프로젝트를 생성할 때 승인이 필요한 API이다.)
4) API 및 서비스 > 사용 설정된 API 및 서비스에서 맨밑으로 스크롤을 내려 Google Analytics Data API가 있는지 확인한다.
5) 상단 구글 클라우드 로고 우측 버튼을 클릭하 새 프로젝트를 생성한다.
6) 프로젝트 이름을 설정하고, '만들기'를 클릭한다.
7) 새로 만든 프로젝트를 클릭한다. 그러면 Google Analytics Data API가 없는 것으로 확인 된다.
8) 상단의 '+ API 및 서비스 사용 설정'을 클릭한다.
9) Google Analytics Data API를 검색하여 '사용'을 클릭한다.
10) 사용자 인증 정보 탭을 클릭한 후, 상단 '+ 사용자 인증 정보 만들기'를 클릭한다.
11) '서비스 계정'을 클릭한다.
12) 서비스 계정이름을 설정하고, '완료'를 클릭한다.
13) 서비스 계정에 새로 생긴 이메일을 클릭후, 복사한다.
14) 상단 '키' 탭을 클릭 후, '키 추가' 클릭, '새 키 만들기'를 클릭한다.
15) JSON 키 유형으로 설정 후, '만들기' 클릭

### GA4 선행 작업
1) https://analytics.google.com/analytics/web/ 에서 설정 > 계정 > 계정 엑세스 관리 클릭
2) +버튼 클릭 후, 사용자 추가 클릭
3) 상단 13번에서 복사해놓았던 이메일 주소를 붙여넣고, 뷰어 권한으로 추가
4) 속성 > 속성 세부정보에서 속성 ID 복사

In [2]:
import numpy as np
import pandas as pd
import os

#### GA4 API 관련 라이브러리 불러오기

In [3]:
# Google Analytics Data API의 클라이언트를 초기화합니다. 이를 통해 API 호출을 수행할 수 있습니다.
from google.analytics.data_v1beta import BetaAnalyticsDataClient 

from google.analytics.data_v1beta.types import (
    RunReportRequest, # Google Analytics Data API에서 보고서를 실행하는 데 필요한 요청 파라미터를 설정합니다. 이 객체에는 날짜 범위, 차원, 메트릭 등이 포함됩니다.
    Dimension, # 분석 보고서에서 측정기준을 나타냅니다. 예를 들어, 'country' 또는 'browser'와 같은 값을 사용할 수 있습니다.
    Metric, # 측정항목을 정의합니다. 예를 들어, 'activeUsers' 또는 'sessions'와 같은 값을 사용할 수 있습니다.
    OrderBy, # 보고서 결과를 정렬하는 방법을 지정합니다. 예를 들어, 특정 메트릭을 기준으로 내림차순 또는 오름차순으로 정렬할 수 있습니다.
    DateRange, # 보고서에 포함될 날짜 범위를 정의합니다. 예를 들어, 특정 기간 동안의 데이터를 요청할 수 있습니다.
)

#### 환경 변수 설정

In [ ]:
# os 모듈을 사용하여 환경 변수를 설정할 수 있습니다.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'GCP에서_생성한_json_파일.json'
# Google Analytics 4 속성 ID 설정
property_id = 'GA4_속성_ID'
# Google Analytics Data API 클라이언트 초기화. 이 클라이언트를 통해 API에 요청을 보내고 데이터를 받을 수 있습니다.
client = BetaAnalyticsDataClient()

#### 측정기준, 측정항목, 날짜 범위 등을 설정

In [33]:
request = RunReportRequest(
        property='properties/'+property_id, # Google Analytics 속성 ID 지정
        dimensions=[Dimension(name="month"),  # 분석 보고서에서 사용할 측정기준들을 정의
                    Dimension(name="sessionMedium")],
        metrics=[Metric(name="averageSessionDuration"), # 분석 보고서에서 사용할 측정항목들을 정의
                 Metric(name="activeUsers")],
        order_bys = [OrderBy(dimension = {'dimension_name': 'month'})], # 보고서 결과를 정렬하는 기준 설정
        date_ranges=[DateRange(start_date="2024-01-01", end_date="today")], # 보고서의 날짜 범위 설정
    )

request

property: "properties/GA4_속성_ID"
dimensions {
  name: "month"
}
dimensions {
  name: "sessionMedium"
}
metrics {
  name: "averageSessionDuration"
}
metrics {
  name: "activeUsers"
}
date_ranges {
  start_date: "2024-01-01"
  end_date: "today"
}
order_bys {
  dimension {
    dimension_name: "month"
  }
}

#### 데이터 불러오기
- client: 이전에 초기화된 BetaAnalyticsDataClient 객체입니다. 이 객체는 Google Analytics Data API와 상호작용하는 데 사용됩니다.
- run_report(request): client 객체의 메서드로, request를 인자로 받아 API에 보고서 데이터를 요청합니다.
- request: RunReportRequest 객체로, 요청하려는 보고서의 날짜 범위, 차원, 메트릭 등을 정의합니다.

In [31]:
# 응답 받기(run_report 메서드)
response = client.run_report(request)

#### 데이터 프레임으로 만들기

In [24]:
data = [] # 데이터를 저장할 빈 리스트

# 응답의 각 행을 반복하여 처리
for row in response.rows:
    # 측정기준(dimension) 값을 동적으로 추출하여 딕셔너리 형태로 저장
    row_data = {dim.name: row.dimension_values[i].value for i, dim in enumerate(response.dimension_headers)}
    # 메트릭(metric) 값도 동적으로 추출하여 딕셔너리에 추가 (문자열을 float로 변환)
    row_data.update({metric.name: float(row.metric_values[i].value) for i, metric in enumerate(response.metric_headers)})
    # 각 행의 데이터를 data 리스트에 추가
    data.append(row_data)
# data 리스트를 데이터프레임으로 변환
df = pd.DataFrame(data)
df.head(1)

,month,sessionMedium,averageSessionDuration,activeUsers
0,01,pcpc,3422.634525,2.0
